##### Reference https://promptapi.com/blog/article/build-your-own-resume-parser-using-python-and-nlp
##### API for skills https://api.emsidata.com/

## Importing libraries

In [166]:
# !pip install pdfminer.six
# !pip install docx2txt
# !pip install pdf2image

In [2]:
import docx2txt
import nltk

In [3]:
import re
import pandas as pd

In [4]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "D:/Program Files/Tesseract-OCR/tesseract"

In [5]:
from pdfminer.high_level import extract_text
# from pdf2image import convert_from_path

In [6]:
import requests
import json

## downloading required nltk packages

In [7]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_cahunker')
# nltk.download('words')

## Extract text from docx

In [8]:
def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        return txt.replace('\t', ' ')
    return None

## Extract text from image

In [9]:
def extract_text_from_image(img_path):
    txt = pytesseract.image_to_string(img_path)
    if txt:
        return txt.replace('\t', ' ')
    return None

## Extract text from pdf

In [10]:
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

In [11]:

# def find_gender(name):
#     url = "https://api.genderize.io/?name="
#     n = name
#     r = requests.get(url+n).json()
#     gender =  r["gender"]
#     if gender:
#         return gender
#     else:
#         return "Cannot identify gender"
    

## Extract phone numbers

In [12]:
def extract_phone_number(resume_text):
    PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
    phone = re.findall(PHONE_REG, resume_text)
    if phone:
        number = ''.join(phone[0])

        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None


## Extract emails

In [13]:
def extract_emails(resume_text):
    EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
    return re.findall(EMAIL_REG, resume_text)

## Extract skills


In [27]:
def extract_skills(input_text):
    SKILLS_DB = [
    'machine learning',
    'data science',
    'python',
    'word',
    'excel',
    'English']
    
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # we create a set to keep the results in.
    found_skills = set()

    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)
    
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills


## Extract skills v2.0

In [14]:
def extract_skill_list(text):
    CLIENT_ID = "f6hl2k37jym2ahm5"
    CLIENT_SECRET= "L1VwGoxH"
    Scope = "emsi_open"

    url = "https://auth.emsicloud.com/connect/token"
    payload = f"client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&grant_type=client_credentials&scope=emsi_open"
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    res = requests.request("POST", url, data=payload, headers=headers).json()
    token = res["access_token"]

    url = "https://emsiservices.com/skills/versions/latest/extract"

    data = {"text": text,
            "confidenceThreshold": 0.6}
    headers = {
        'Authorization': f"Bearer {token}",
        'Content-Type': "application/json"
        }

    response = requests.request("POST", url, data=json.dumps(data), headers=headers)

    skill_json = response.json()

    skill_set = []
    for i in range(len(skill_json["data"])):
        skill_set.append(skill_json["data"][i]["skill"]["name"])

    return skill_set

## Display result

In [16]:
def show_result(text):
#     text = extract_text_from_docx('Input/Resume/doc/resume.docx')
    skills = extract_skill_list(text)
    phone_number = extract_phone_number(text)
    email = extract_emails(text)
    print(f"""
    Email : {email[0]}  
    Phone number : {phone_number}
    skills : {skills}
    """)

In [17]:
text = extract_text_from_docx('Input/Resume/doc/resume.docx')

In [18]:
text_img = extract_text_from_image("Input/Resume/image/resume.jpg")

In [19]:
text_pdf = extract_text_from_pdf("Input/Resume/pdf/resume.pdf")

In [20]:
text_doc = extract_text_from_docx("Input/Resume/doc/Software-Engineer-Resume.docx")

In [21]:
print(text)

MARIA LISA

Email: maria123@gmail.com

Phone: (555)-555-5555



CAREER SUMMARY

Around 3 Years of experience in Analysis, Design, Development, Management and Validation of various stand-alone and client-server applications.

2+ years of experience in implementing Object-Oriented Python, Hash Tables (Dictionaries) and Multi threading.

Experience in installing software using pip command for python libraries and extensive usage of the PEP8 coding convention.

Good understanding on Django ORM and SQLAlchemy.

Experienced in interfacing with third party API’s using REST and SOAP

Worked with varieties of Relational Databases (RDBMS) like SQLite, MySQL and PostgreSQL.

Experienced in writing SubQueries, Triggers, Cursors, Stored Procedures and Functions.

Proficient knowledge of Web services like Google Cloud and Amazon AWS Services.

Good Knowledge on Linux and Shell Scripting.

Experienced in developing web-based applications using Python, PHP, XML, JSON and MVC3

Well versed in design an

In [22]:
show_result(text)


    Email : maria123@gmail.com  
    Phone number : (555)-555-5555
    skills : ['Software Design', 'Relational Databases', 'Object-Oriented Programming (OOP)', 'MySQL', 'Data Exchange', 'Bootstrap (Front-End Framework)', 'C++ (Programming Language)', 'JavaScript (Programming Language)', 'Development Management', 'Database Management Systems', 'Application Programming Interface (API)', 'Pandas (Python Package)', 'Microsoft Visual Studio', 'Shell Script', 'Scikit-learn (Machine Learning Library)', 'Extensible Markup Language (XML)', 'PostgreSQL', 'Linux', 'Amazon Web Services', 'Stored Procedure', 'Algorithms', 'SQLAlchemy', 'PHP (Scripting Language)', 'HTML5', 'Python (Programming Language)', 'SQL (Programming Language)', 'Model View Controller', 'Seaborn', 'Ajax (Programming Language)', 'Web Servers', 'Django (Web Framework)', 'Eclipse (Software)', 'Extensible HyperText Markup Language (XHTML)', 'Unit Testing', 'SQLite', 'Angular (Web Framework)', 'MongoDB', 'Apache Subversion', 'Jav

In [23]:
show_result(text_img)


    Email : reynold.stamm80@gmail.com  
    Phone number : (400) 223-1058
    skills : ['Asana', 'Power BI', 'Atlassian Bamboo', 'NoSQL', 'Operating Cost', 'Process Architecture', 'Microsoft SQL Servers', 'C++ (Programming Language)', 'Nagios', 'Puppet (Configuration Management Tool)', 'Microsoft Azure', 'ECMAScript 2015', 'Software Design Patterns', 'Github', 'Flask (Web Framework)', 'ECMAScript 2017', 'Storages', 'Git (Version Control System)', 'Capistrano (Software)', 'Scrum (Software Development)', 'GraphDB', 'SQLAlchemy', 'Python (Programming Language)', 'Full Stack Software Engineering', 'Django (Web Framework)', 'Codeship', 'Solid Principles', 'Application Programming Interface (API)', 'PL/SQL', 'Bitbucket', 'Code Complexity', 'Apache Subversion', 'Machine Learning', 'JIRA', 'Ansible', 'Atlassian Crucible', 'JRuby', 'IBM Cloud Computing', 'Load Testing', 'Natural Language Processing', 'Multicast Address Allocation Server', 'Willingness To Learn', 'User Experience', 'DevOps', 'S

In [24]:
show_result(text_pdf)


    Email : info@qwikresume.com  
    Phone number : (123) 456 78 99
    skills : ['MySQL', 'Amazon Elastic Compute Cloud', 'Registered Investment Advisor', 'Application Programming Interface (API)', 'Autoscaling', 'Virtual Local Area Network (VLAN)', 'Github', 'Requirements Analysis', 'Git (Version Control System)', 'Hypervisor', 'OpenStack', 'Bugzilla', 'Scrum (Software Development)', 'Kernel-Based Virtual Machine', 'Heroku', 'Python (Programming Language)', 'Django (Web Framework)', 'Continuous Integration', 'Amazon Simple Notification Service (SNS)', 'Architectural Design', 'JIRA', 'Lightweight Directory Access Protocols', 'Model View Controller', 'Gerrit', 'Jenkins', 'Version Control', 'Firewall', 'Cascading Style Sheets (CSS)', 'Software Engineering', 'JavaScript (Programming Language)', 'Bash (Scripting Language)', 'AWS Lambda', 'Swift (Programming Language)', 'Management', 'Concurrent Versions System (Software)', 'Automation', 'Troubleshooting (Problem Solving)', 'Urllib', 'Da

In [25]:
show_result(text_doc)


    Email : youremail@gmail.com  
    Phone number : 895-555-5555
    skills : ['MySQL', 'Bash (Scripting Language)', 'Application Programming Interface (API)', 'Vue.js', 'LAMP (Software Bundle)', 'Network Planning And Design', 'Linux Servers', 'Docker (Software)', 'Linux', 'Cascading Style Sheets (CSS)', 'PHP (Scripting Language)', 'Python (Programming Language)', 'RESTful API', 'Full Stack Software Engineering', 'Windows Servers', 'React.js', 'Kohana', 'Systems Architecture', 'C# (Programming Language)', 'Elasticsearch', 'Node.js', 'SQL (Programming Language)', 'Consulting', 'Laravel', 'Hyper-V', 'HTML5', 'Amazon Web Services', 'JQuery', 'MariaDB', 'Software Engineering', 'JavaScript (Programming Language)', 'SAS (Software)', 'Management', 'Back End (Software Engineering)', 'Microsoft Access', 'Web Applications', 'R (Programming Language)', 'Leadership', 'Web Development']
    
